# 第一部分 课程内容

* 教学目的

    本节教程将介绍Fama-French的三因子模型，以及实际的应用。*函数*是python的重要组成部分，但是在notebook中很少被使用到，这个教程将展现函数的力量。在编程的过程中使用到python的自函数，可以方便我们重用代码，节约时间。
    
    三因子模型的提出是基于美国股市历史回报率的实证研究结果，目的在于解释股票市场的平均回报率受到哪些风险溢价因素的影响，在CAPM模型（资本资产定价模型）基础上，增加了市净率和公司规模对平均回报率影响的考虑。
    
* 教学材料

    我们将针对中证500成份股从2007.6到2016.5的数据进行实证研究，以验证三因子模型的有效性。
    
    我们要使用一些数据分析和画图的基础包，并且使用时间处理包CAL。


## 1 Fama-French 三因子模型基本原理
### 1.1 三因子模型具体形式

Fama-French三因子模型是量化领域最经典的模型之一，该模型的提出是在论文《commom risk factors in returns on bonds and stocks》里。在资本资产定价模型（CAPM）等传统理论下，投资组合的全部风险溢价由一个系数表示。但是这一模型在解释股票市场回报的现实情况上，遇到了诸多困难，三因子模型通过引入两个新的解释变量：市净率、公司规模，与CAPM中的市场指数一同估计股票的回报水平，即：

$$
E(R(t))=R_f(t)+\beta E(R_M(t)-R_f(t))+sE(SMB(t))+hE(HML(t))+\alpha
$$

$R_f$是市场无风险收益率,$R_M$是市场组合的收益率，$R_M-R_f$表示的是市场因子，$SMB$表示的是规模(市值)因子，$HML$表示账面市值比因子，即市净率溢价，$\alpha$是超额收益率，在理想的情况下，投资组合的超额回报将全部被三因素解释，从而$\alpha$ 应在统计学意义上等于0.。

### 1.2 Black-Jensen-Scholes时间序列回归

横截面回归大家都非常熟悉，无论是单变量还是多变量回归，都是在研究解释变量对响应变量的解释能力。Black-Jensen-Scholes时间序列回归的方法是Black,Jensen和Scholes所提出来验证CAPM的。早期的验证方法是先使用一个单只股票的时间序列回归估计参数，再用横截面回归验证CAPM推出的假设。但是这样回归会有问题，时间序列回归则是提出的改进算法，即根据前一期估计的贝塔值对股票排序再进行分组，分别估计各投资组合的参数，每五年重新估计，然后检验各个投资组合的阿尔法是否显著为0。
$$
E(R(t))=R_f(t) +\beta(E(R_M(t))-R_f(t))+\alpha
$$

### 1.3 解释变量
解释变量就是我们需要验证的三个因子，市场超额收益，规模和账面市值比。我们要按照论文里的思路对其进行处理。

1. 把股票按每年5月末时的市值（size）大小进行排序，按照50%分位值把股票分为S(small)和B(big)两组；
2. 再依据5月末时的账面市值比（我们取1/PB）大小对500只股票进行排序，分为L（low，30%）,M(medium,40%),H(high,30%)三组;
3. 再分别对S,B和L,M,H取交集，股票即被分为了SL,SM,SH,BL,BM,BH六组。也就是说，分组每年5月末进行一次，800只股票每次被重新分为了SL,SM,SH,BL,BM,BH六组，前一年6月到第二年5月重新分组时的投资组合都是一样的

这里为什么要按市值分为两组，按账面市值比分为三组呢？
是因为账面市值比有更强的作用，我们要把它分得更细。

### 1.4 因子
市值因子：
$$
SMB = 1/3*(SL+SM+SH)-1/3*(BL+BM+BH)
$$
表示的是由于公司规模不同造成的风险溢价

账面市值比因子:
$$
HML = (SH+BH)/2-(SL+BL)/2
$$
表示由于账面市值比不同造成的风险溢价

可以看出因子的值是一个市值加权月收益率序列,我们研究了九年的数据，所以因子的长度是9*12=108

In [55]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
# 画图包
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot')
import seaborn as sns

import tushare as ts
import datetime
# from CAL.PyCAL import *

由于不知道每年五月末最晚的交易时间是多久，所以要分开计算，即每一段交易时间是从6.1到次年的5.31。这里已经把数据提取好了。

In [58]:
data = pd.read_csv('../data/10years.csv')
data.head()

,ticker,20080530Value,20080530PB,20090601Value,20090601PB,20100601Value,20100601PB,20110601Value,20110601PB,20120601Value,20120601PB,20130531Value,20130531PB,20140530Value,20140530PB,20150601Value,20150601PB
0,1,5.786266e+10,4.1209,5.682944e+10,3.3288,5.372400e+10,2.4299,6.074379e+10,1.6897,8.084647e+10,1.0525,1.075391e+11,1.2116,1.094886e+11,0.9334,2.275080e+11,1.6642
1,2,1.357221e+11,4.4936,1.117113e+11,3.4172,7.872571e+10,2.0197,8.598254e+10,1.8884,1.006062e+11,1.8508,1.318610e+11,2.0107,9.417798e+10,1.2806,1.643592e+11,1.8493
2,6,5.863039e+09,3.0026,5.979691e+09,3.3847,4.016891e+09,1.8770,5.424325e+09,2.1687,5.993385e+09,2.0584,6.428548e+09,1.8133,6.358477e+09,1.5555,2.257192e+10,5.2050
3,9,7.624866e+09,4.5045,1.050237e+10,5.2812,1.034032e+10,4.5485,1.772470e+10,6.7849,1.235820e+10,4.6178,1.414703e+10,4.7051,1.280705e+10,3.9786,3.343426e+10,8.9712
4,12,2.229807e+10,5.3060,2.241631e+10,4.8493,2.369584e+10,4.2105,3.396570e+10,4.9919,2.013075e+10,2.8628,2.291170e+10,3.3129,1.485940e+10,1.8166,3.349591e+10,3.9451


为了方便使用，以及为了代码整洁意义清晰，我们需要在python里自己定义方法，比如下面，定义了一个进行分组的函数get_6groups，方便对每年的数据进行分组，这样就可以把分组的逻辑和处理数据的逻辑分离开，让主体逻辑更加清晰简单，并且我们可以在一个循环里，重复使用这个方法，用同样的处理逻辑，处理不同的年份数据。

In [65]:
# 传入的数据总是三列 ticker,Value,PB
def get_6groups(data):
    ME=pd.DataFrame()
    ME['ticker']=data.iloc[:,0]
    ME['marketValue']=data.iloc[:,1]
    BP=pd.DataFrame()
    BP['ticker']=data.iloc[:,0]
    BP['marketValue']=data.iloc[:,2]
    
    ME50=np.percentile(ME['marketValue'],50)                                     #算出市值大小的50%分位值
    S=ME[ME['marketValue']<=ME50]['ticker'].tolist()                                #按市值大小分为两组，存为列表
    B=ME[ME['marketValue']>ME50]['ticker'].tolist()
    BP=BP[BP>0].dropna()                                                  #去掉PB值为负的股票
    BP[['PB']]=1/BP[['PB']]                                                #取1/PB，为账面市值比
    BP30=np.percentile(BP['PB'],30)
    BP70=np.percentile(BP['PB'],70)
    L=BP[BP['PB']<=BP30]['ticker'].tolist()                                      #按1/PB大小分为三组
    H=BP[BP['PB']>BP70]['ticker'].tolist()
    M=list(set(BP['ticker'].tolist()).difference(set(L+H)))
    SL=list(set(S).intersection(set(L)))                                       #对S组和L组的股票取交集，作为SL组的股票组合
    SM=list(set(S).intersection(set(M)))
    SH=list(set(S).intersection(set(H)))
    BL=list(set(B).intersection(set(L)))
    BM=list(set(B).intersection(set(M)))
    BH=list(set(B).intersection(set(H)))
    return SL,SM,SH,BL,BM,BH

下面我们要计算每个投资组合的月收益率，计算投资组合的月收益率时，要算市值加权的收益率，这是为了最小化方差（风险）

In [ ]:
# 得到投资组合x从Year的6月到Year+1的5月的月收益率序列
def get_returnMonthly(x,Year):
    #先用交易日日历得到Year的5月到Year+1的5月的月末交易日日期
    data=DataAPI.TradeCalGet(exchangeCD=u"XSHG",beginDate=str(Year*10000+501),endDate=str((Year+1)*10000+601),field=['calendarDate','isMonthEnd'])
    data = data[data['isMonthEnd'] == 1]
    date= map(lambda x: x[0:4]+x[5:7]+x[8:10], data['calendarDate'].values.tolist())
    # 调用投资组合x每只股票每个月末的市值，收盘价用来计算收盘价
    returnMonthly=np.zeros(12)
    for i in range(12):
        inf1=DataAPI.MktEqudAdjGet(tradeDate=date[i],ticker=x,field=u"ticker,closePrice").set_index('ticker')  #前一个月的收盘价
        inf2=DataAPI.MktEqudAdjGet(tradeDate=date[i+1],ticker=x,field=u"ticker,marketValue,closePrice").set_index('ticker')   #当月的收盘价和市值
        Return=pd.concat([inf2,inf1],axis=1)
        Return.columns=['Weight','Return','WReturn']                   #计算每只股票收益率和市值加权的权重以及两者的乘积
        Return['Weight']=Return['Weight']/Return['Weight'].sum()
        Return['Return']=Return['Return']/Return['WReturn']-1
        Return['WReturn']=Return['Weight']*Return['Return']
        returnMonthly[i]=Return['WReturn'].sum()
    return returnMonthly

In [66]:
# 计算每年的SMB和HML因子，合在一起
SMB=[]
HML=[]
r_groups=pd.DataFrame()  #用于存储每个组合的月收益率序列，方便我们之后查看
r_groups['SL']=np.zeros(108)
r_groups['SM']=np.zeros(108)
r_groups['SH']=np.zeros(108)
r_groups['BL']=np.zeros(108)
r_groups['BM']=np.zeros(108)
r_groups['BH']=np.zeros(108)

idx = 1
while idx < data.shape[1]:
    one=pd.DataFrame()
    one['ticker']=data.iloc[:,0]
    one['marketValue']=data.iloc[:,idx]
    one['PB']=data.iloc[:,idx+1]
    #依据当年5月末的市值和账面市值比分组
    SL,SM,SH,BL,BM,BH=get_6groups(one)
    idx = idx + 1
    print(SL,SM,SH,BL,BM,BH)

KeyError: "None of [Index(['PB'], dtype='object')] are in the [columns]"

类似的方法可以得到当年5月末到次年的市值加权月收益率序列，在此不列举了，最终的结果如下

In [ ]:
SMB=np.array(SMB)
HML=np.array(HML)

In [ ]:
data=DataAPI.TradeCalGet(exchangeCD=u"XSHG",beginDate=str(2007*10000+501),endDate=str((2015+1)*10000+601),field=['calendarDate','isMonthEnd'])
data = data[data['isMonthEnd'] == 1]
date= map(lambda x: x[0:4]+x[5:7]+x[8:10], data['calendarDate'].values.tolist())
r_groups.index=date[1:]
r_groups.plot(figsize=[12,7])

市场因子：
$$
R_M-R_f
$$
$R_M$取的就是中证800指数的收益，$R_f$取的是银行间质押式回购利率_同业拆借中心R007

In [ ]:
#先用交易日日历得到Year的5月到Year+1的5月的月末交易日日期

data=DataAPI.TradeCalGet(exchangeCD=u"XSHG",beginDate='20070501',endDate='20160601',field=['calendarDate','isMonthEnd'])
data = data[data['isMonthEnd'] == 1]
date = map(lambda x: x[0:4]+x[5:7]+x[8:10], data['calendarDate'].values.tolist())
RmMonthly=np.zeros(108)
RfMonthly=np.zeros(108)
for i in range(108):
    index1=DataAPI.MktIdxdGet(tradeDate=date[i],indexID=u"000906.ZICN",field=u"closeIndex") #上月指数收盘
    index2=DataAPI.MktIdxdGet(tradeDate=date[i+1],indexID=u"000906.ZICN",field=u"closeIndex")  #当月指数收盘
    RmMonthly[i]=index2['closeIndex'][0]/index1['closeIndex'][0]-1
    rf=DataAPI.MktIborGet(ticker=u"Shibor1D",beginDate=date[i+1],endDate=date[i+1],field=u"rate")  #当月无风险收益
    RfMonthly[i]=rf['rate'][0]/100/12    #给出的是年化无风险收益，这里需要转化成月的
MF=RmMonthly-RfMonthly  #市场因子

三个因子我们都得到了，再来看看三个因子的状况：

In [ ]:
factor=pd.DataFrame()
factor['MF']=MF
factor['SMB']=SMB
factor['HML']=HML
factor.index=date[1:]
factor.plot(figsize=[12,7])
factor.describe()

到这里，我们的三个因子就处理完了，三个解释变量都做成了数组，下面可以分析一下各个因子之间的相关系数

In [ ]:
x=np.zeros((3,108))
x[0]=MF
x[1]=SMB
x[2]=HML
Correlations=pd.DataFrame(np.corrcoef(x))
Correlations.columns=['MF','SMB','HML']
Correlations.index=['MF','SMB','HML']
Correlations

### 1.5 响应变量
首先我们将股票按之前的方法分为25个组合，在每年5月末，按照市值大小将股票排序并分为5组，然后按照账面市值比大小把股票分为5组，交叉取交集，得到5\*5=25个股票组合。

也就是说，我们将作25个回归，每次回归时的解释变量都一样，响应变量不同。

In [ ]:
#每年5月末，按照市值大小将股票排序分为5组
def get_25groups(breakpoint):
    universe = set_universe('000906.ZICN',date=breakpoint)
    C=DataAPI.MktEqudGet(ticker='000028',beginDate=str(int(breakpoint)-20),endDate=breakpoint,field=u"ticker,tradeDate")
    breakpoint=filter(lambda x:x.isdigit(),C.iat[len(C)-1,1])                             #取breakpoint前最近一个交易日日期
    ME=DataAPI.MktEqudGet(tradeDate=breakpoint,secID=universe,field=u"ticker,marketValue").dropna()
    MEq=np.zeros(6) #用于存储ME的分位值
    ME_5=pd.Series([]) #用于存储依据市值分好的5个组
    BP=DataAPI.MktStockFactorsOneDayGet(tradeDate=breakpoint,secID=universe,field=u"ticker,PB").dropna() 
    BP=BP[BP>0].dropna()                                                  #去掉PB值为负的股票
    BP[['PB']]=1/BP[['PB']]                                               #取1/PB，为账面市值比
    BPq=np.zeros(6)  #用于存储1/PB的分位值
    BP_5=pd.Series([]) #用于存储依据账面市值比分好的5个组
    for i in range(5):
        MEq[i+1]=np.percentile(ME['marketValue'],(i+1)*20)                           #算出市值大小的20%,40%,60%,80%分位值
        BPq[i+1]=np.percentile(BP['PB'],(i+1)*20)
        D=ME[(ME['marketValue']>MEq[i]) & (ME['marketValue']<=MEq[i+1])]['ticker'].tolist()     #取市值处于相应分位值区间的股票
        ME_5 = pd.concat([ME_5,pd.Series(D)],axis=1) if not ME_5.empty else pd.Series(D)        #存于dataframe里
        E=BP[(BP['PB']>BPq[i]) & (BP['PB']<=BPq[i+1])]['ticker'].tolist()
        BP_5 = pd.concat([BP_5,pd.Series(E)],axis=1) if not BP_5.empty else pd.Series(E)
    ME_5.columns=range(5) #重命名列名
    BP_5.columns=range(5)
    Group25=pd.Series([]) #用于存著交叉取交集后的25个股票组合
    for i in range(5):
        for j in range(5):
            s1=ME_5[i].dropna().tolist()
            s2=BP_5[j].dropna().tolist() 
            stocks=pd.Series(list(set(s1).intersection(set(s2)))) #取交集
            Group25 = pd.concat([Group25,stocks],axis=1) if not Group25.empty else stocks
    Group25.columns=range(25)
    return Group25

计算25个股票组合，每个组合的市值加权月收益率序列

In [ ]:
EReturn=np.zeros((25,12*9)) #用于存储25个组合的超额收益序列
for i in range(25):
    a=[]
    for Year in [2007,2008,2009,2010,2011,2012,2013,2014,2015]:
        Group25=get_25groups(str(Year*10000+531))   #每年进行分组
        a=a+(get_returnMonthly(Group25[i].dropna().tolist(),Year)).tolist()   #收益率转化为list，方便每年相加
    EReturn[i]=np.array(a)-RfMonthly

我们看一下25个组合平均每年的公司数：

In [ ]:
number=np.zeros((25,9)) 
for i in range(25):
    for j in range(9):
        Group25=get_25groups(str((j+2007)*10000+531))   #每年进行分组
        number[i][j]=len(Group25[i].dropna().tolist())
number_mean=np.zeros(25)
for i in range(25):
    number_mean[i]=number[i].mean()
numbers=pd.DataFrame(number_mean.reshape(5,5))
numbers.columns=['small_BE/ME','1','2','3','big_BE/ME']
numbers.index=['small_size','1','2','3','big_size']
numbers

还可以看一下25个组合平均每年的总市值大小，验证一下分组的正确性：

In [ ]:
MarketValue=np.zeros((25,9)) 
for i in range(25):
    for j in range(9):
        breakpoint=str((j+2007)*10000+531)
        Group25=get_25groups(breakpoint)  #每年进行分组
        C=DataAPI.MktEqudGet(ticker='000028',beginDate=str(int(breakpoint)-20),endDate=breakpoint,field=u"ticker,tradeDate")
        breakpoint=filter(lambda x:x.isdigit(),C.iat[len(C)-1,1])                             #取breakpoint前最近一个交易日日期
        data=DataAPI.MktEqudGet(tradeDate=breakpoint,ticker=Group25[i].dropna().tolist(),field=u"ticker,marketValue").dropna()
        MarketValue[i][j]=data['marketValue'].sum()
MarketValue_mean=np.zeros(25)
for i in range(25):
    MarketValue_mean[i]=MarketValue[i].mean()
MV=pd.DataFrame(MarketValue_mean.reshape(5,5))
MV.columns=['small_BE/ME','1','2','3','big_BE/ME']
MV.index=['small_size','1','2','3','big_size']
MV

上面的股票组合从左到右，账面市值比越来越大；从上往下，市值越来越大，说明我们的分组是正确的

看一下25个组合超额收益的均值和方差：

In [ ]:
EReturn_mean=np.zeros(25)
for i in range(25):
    EReturn_mean[i]=EReturn[i].mean()
mean=pd.DataFrame(EReturn_mean.reshape(5,5))
mean.columns=['small_BE/ME','1','2','3','big_BE/ME']
mean.index=['small_size','1','2','3','big_size']
mean

In [ ]:
EReturn_std=np.zeros(25)
for i in range(25):
    EReturn_std[i]=EReturn[i].std()
std=pd.DataFrame(EReturn_std.reshape(5,5))
std.columns=['small_BE/ME','1','2','3','big_BE/ME']
std.index=['small_size','1','2','3','big_size']
std

## 2 回归和结果
### 2.1 回归一
CAPM回归模型：
$$
R(t)-R_f(t) = a+b(R_M(t)-R_f(t))+e(t)
$$
这就是经典的CAPM模型，我们可以检验一下它在中国A股市场的有效性：

In [ ]:
#作25次回归
a1=np.zeros(25)   #a项
b1=np.zeros(25)   #市场因子项系数
e1=np.zeros((25,108))   #残差项
R2_1=np.zeros(25)   #R2相关系数平方
tb1=np.zeros(25)
ta1=np.zeros(25)

ap1=np.zeros(25)  #a显著性检验的P值，下面类同
bp1=np.zeros(25)
for i in range(25):
    X=np.zeros((1,108))
    X[0]=MF
    X=X.T
    X = sm.add_constant(X,has_constant='skip')
    y=EReturn[i]
    model = sm.OLS(y, X)
    results = model.fit()
    a1[i] = results.params[0]
    b1[i] = results.params[1]
    ap1[i]=results.pvalues[0]
    bp1[i]=results.pvalues[1]
    R2_1[i] = results.rsquared
    e1[i] = results.resid
    tb1[i] = results.tvalues[1]
    ta1[i] = results.tvalues[0]

先看一下这25个回归的判定系数$R^2$，它度量了拟合程度的好坏。

In [ ]:
R2inf1=pd.DataFrame(R2_1.reshape(5,5))
R2inf1.columns=['small_BE/ME','1','2','3','big_BE/ME']
R2inf1.index=['small_size','1','2','3','big_size']
R2inf1

25个回归的R2大多处于0.7~0.9之间，已经是比较好的结果了，这点可以和其它的回归模型对比。

下面看一下市场因子的系数$\beta$：

In [ ]:
binf1=pd.DataFrame(b1.reshape(5,5))
binf1.columns=['small_BE/ME','1','2','3','big_BE/ME']
binf1.index=['small_size','1','2','3','big_size']
binf1

我们可以看到$\beta$大多处于1左右。下面我们来检验其显著性，回归系数的显著性检验用的是t检验：原假设为t=0,若t统计量的值大于给定显著水平下的t分位值，则拒绝原假设，说明该系数显著大于0

In [ ]:
t107=stats.t.isf(0.025,106)  #自由度为n-p，显著水平5%下的t分位值
t107

In [ ]:
tbinf1=pd.DataFrame(tb1.reshape(5,5))
tbinf1.columns=['small_BE/ME','1','2','3','big_BE/ME']
tbinf1.index=['small_size','1','2','3','big_size']
tbinf1

我们可以看到所有回归里$\beta$的t统计量的值都大于临界值，我们应该拒绝原假设，即表明$\beta$系数显著

以上，说明资本资产定价模型是有效的，市场因子的影响是显著的

### 2.2 回归二
如上，我们检验了CAPM模型的有效性，现在我们不妨检验一下另外两个因子对股票超额收益的解释作用。
$$
R(t)-R_f(t) = a+sSMB(t)+hHML(t)+e(t)
$$
$SMB$和$HML$分别代表规模(市值)因子和账面市值比因子。

In [ ]:
#作25次回归

a2=np.zeros(25)   #a项
s2=np.zeros(25)   #规模因子项系数
h2=np.zeros(25)   #账面价值比项系数
e2=np.zeros((25,108))   #残差项
R2_2=np.zeros(25)   #R2相关系数平方
ta2=np.zeros(25)
tb2=np.zeros(25)

ap2=np.zeros(25)  #a显著性检验的P值，下面类同
sp2=np.zeros(25)
hp2=np.zeros(25)
for i in range(25):
    X=np.zeros((2,108))
    X[0]=SMB
    X[1]=HML
    X=X.T
    X = sm.add_constant(X,has_constant='skip')
    y=EReturn[i]
    model = sm.OLS(y, X)
    results = model.fit()
    ap2[i]=results.pvalues[0]
    sp2[i]=results.pvalues[1]
    hp2[i]=results.pvalues[2]
    a2[i] = results.params[0]
    s2[i] = results.params[1]
    h2[i] = results.params[2]
    R2_2[i] = results.rsquared
    e2[i] = results.resid
    tb2[i] = results.tvalues[1]
    ta2[i] = results.tvalues[0]

同样，我们看一下25个回归的判定系数R2的情况：

In [ ]:
R2inf2=pd.DataFrame(R2_2.reshape(5,5))
R2inf2.columns=['small_BE/ME','1','2','3','big_BE/ME']
R2inf2.index=['small_size','1','2','3','big_size']
R2inf2

从R2可以看到，基本都在0.5以下，这个回归的结果比回归一差了很多，这个模型并不好,也就是说只用市值因子和账面市值比因子来解释股票超额收益是不合适的

### 2.3 回归三
这里的回归模型就是我们经典的三因子模型

$$
R(t)-R_f(t) = \alpha+b(R_M(t)-R_f(t))+sSMB(t)+hHML(t)+e(t)
$$

$R(t)-R_f(t)$：市场因子

$SMB(t)$：规模(市值)因子

$HML(t)$ ：账面市值比因子

In [ ]:
#作25次回归
a=np.zeros(25)   #a项
b=np.zeros(25)   #市场因子项系数
s=np.zeros(25)   #规模因子项系数
h=np.zeros(25)   #账面价值比项系数
e=np.zeros(25)   #残差项
R2=np.zeros(25)   #R2相关系数平方
for i in range(25):
    x=np.zeros((3,108))
    x[0]=MF
    x[1]=SMB
    x[2]=HML
    y=EReturn[i]
    x=np.mat(x).T
    y=np.mat(y).T
    regr = linear_model.LinearRegression()
    regr.fit(x,y)
    b[i]=regr.coef_[0][0]
    s[i]=regr.coef_[0][1]
    h[i]=regr.coef_[0][2]
    a[i]=regr.intercept_[0]
    e[i]=regr.residues_
    R2[i]=regr.score(x,y)

In [ ]:

ap=np.zeros(25)  #a显著性检验的P值
ta=np.zeros(25)
tb=np.zeros(25)
ts=np.zeros(25)
th=np.zeros(25)
for i in range(25):
    X=np.zeros((3,108))
    X[0]=MF
    X[1]=SMB
    X[2]=HML
    X=X.T
    X = sm.add_constant(X,has_constant='skip')
    y=EReturn[i]
    model = sm.OLS(y, X)
    results = model.fit()
    ap[i]=results.pvalues[0]
    ta[i] = results.tvalues[0]
    tb[i] = results.tvalues[1]
    ts[i] = results.tvalues[2]
    th[i] = results.tvalues[3]    

我们先看一下回归的$R^2$：

In [ ]:
R2inf3=pd.DataFrame(R2.reshape(5,5))
R2inf3.columns=['small_BE/ME','1','2','3','big_BE/ME']
R2inf3.index=['small_size','1','2','3','big_size']
R2inf3

我们可以看到$R^2$基本上都在0.9以上，三因子模型的拟合程度非常好，说明三因子模型是比CAPM更有效的模型

### 2.4 回归结果分析
如上三因子模型的有效性已经得到验证，确实能够解释股票收益来源，那么A股市场对这些因子有什么偏好呢？比如：长期来看，小盘股跑赢大盘股，还是大盘股能跑赢小盘股呢？

我们可以从第一个回归(CAPM)的截距项找到答案：

In [ ]:
ainf1=pd.DataFrame(a1.reshape(5,5))
ainf1.columns=['small_BE/ME','1','2','3','big_BE/ME']
ainf1.index=['small_size','1','2','3','big_size']
tainf1=pd.DataFrame(ta1.reshape(5,5))
tainf1.columns=['small_BE/ME','1','2','3','big_BE/ME']
tainf1.index=['small_size','1','2','3','big_size']
print '第一个回归的a值：'
print ainf1.to_html()
print '自由度为n-p，显著水平5%下的t分位值:'+str(stats.t.isf(0.025,106))
print '第一个回归的a的t检验统计量：'
print tainf1.to_html()

可以看到25个回归里只有最后5个回归的a值是显著为0的，其余a值我们可以认为是不显著为0的，也就是说超额收益没有被完全解释，这也是三因子模型存在的必要。

我们从市值的角度来分析一下，以上结果，从上到下组合的市值越来越大，a值越来越小，a代表的是超额收益，也就是说市值越小的股票越容易获得超额收益，这点和我们的认知相同

从左到右，组合的账面市值比越来越大，a值从趋势上是越来越大，但也有很多反转，也就是说账面市值比越高的组合越容易获得超额收益这个结论并不准确。

## 3.总结
通过对三个模型的回归进行对比，我们可以看到，从$R^2$来看，拟合结果最差的是第二个模型（只用市值因子和账面市值比），拟合结果最好的是第三个模型，也就是fama三因子模型。

综上，我们验证了Fama三因子模型在中国A股市场是有效的，也印证了市值小的股票更容易获得超额收益这一点。

# 第二部分 课程练习

1. 请找到海外市场的部分股票信息，并做相同的分析，看看国内外的市场是否相似
2. 为了熟练python的函数，请针对自己的问题，进行python编程以掌握函数的妙处

# 第三部分 延伸学习

由于容量问题，很多东西是没有办法介绍的，下面给出一些延伸学习的主题，请自行学习。
* 时间处理在数据分析中非常重要，请自行了解如何使用time、datetime等不同的package使用方法
* 比三因子模型更细致的是五因子模型，请去了解他，并比较两者的不同
* Fama也是有效市场假说的提出者，请自主学习这个假说